### Czyszczenie

In [22]:
import codecs
import os
import re
from IPython.core.debugger import set_trace

def clean_and_save(path_rawfile, path_direction):
    f = codecs.open(path_rawfile, encoding='utf-8')
    l = list()
    for line in f:
        l.append(line)
    clean_document = re.sub( '<E(.{1,100})">', '' , l[0]) # ten przedzial dlatego, zeby usuwalo kolejne fragmenty, a nie duzy fragment tekstu
    clean_document = re.sub('</Entity>', '' , clean_document)
    f.close()
    f = codecs.open(path_direction, "w", encoding = 'utf-8')
    f.write(clean_document)
    f.close()

def clean_all_files(absolute_path_source, absolute_path_direction, dirs_list):
    for dirc in dirs_list:
        path_source = absolute_path_source + '\\' + dirc
        path_direction = absolute_path_direction + '\\' + dirc
        for file in os.listdir(path_source):
            clean_and_save(path_source + '\\' + file, path_direction + '\\' + file)

In [ ]:
absolute_path_source = 'C:\\Users\\piotr\\Desktop\\text\\TextMining\\learningData'
absolute_path_direction = 'C:\\Users\\piotr\\Desktop\\text\\TextMining\\cleanData'
dirs = ['korpusGAZETA', 'korpusONET']
clean_all_files(absolute_path_source, absolute_path_direction, dirs)

### Tagowanie

In [34]:
import codecs
from IPython.core.debugger import set_trace
import re
from copy import deepcopy

def flatten_list(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def read_document(file):
    with codecs.open(file, 'r', encoding = 'UTF-8') as f:
        document = f.read()
    return document

def find_entity(doc, entity_forms):
    # For example entity_forms = [ 'Donald Tusk', 'Donalda Tuska'] itp.
    positions = []
    for form in entity_forms:
        positions.append((m.start(), m.end()) for m in re.finditer(form, doc))
    result = flatten_list(positions) 
    result.sort()
    return result

def tag_template(entity_base_form, type_ , category):
    tag_start = '<Entity name="' + entity_base_form + '" type="' + type_ + '" category="' + category + '">'
    tag_end = '</Entity>'
    return (tag_start, tag_end)

def slice_string_and_tag(string, slice_points, tag):
    l = []
    start = 0
    for slice_start, slice_end in slice_points:
        l.append(string[start:slice_start])
        l.append(tag[0])
        l.append(string[slice_start:slice_end])
        l.append(tag[1])
        start = slice_end
    l.append(string[start:-1])
    return ''.join(l)
        
    
def tag_document_base(file, entity_base_form, entity_forms, type_ , category):
    # This one if we search for people
    doc = read_document(file)
    positions = find_entity(doc, entity_forms)
    tag = tag_template(entity_base_form, type_ , category)
    return slice_string_and_tag(doc, positions, tag)
    
def tag_document_model(file, positions, entity_base_form, type_ , category):
    # This one propably better for model (I assume it gives us starting and ending positions)
    doc = read_document(file)
    tag = tag_template(entity_base_form, type_ , category)
    return slice_string_and_tag(doc, positions, tag)

In [35]:
tag_document_base('doc2', 'Marek Edelman', ['Marek Edelman', 'Marka Edelmana'], type_ = 'person', category='lekarz')

'Niesamowita historia książki ze specjalną dedykacją, którą <Entity name="Marek Edelman" type="person" category="lekarz">Marek Edelman</Entity>, legendarny przywódca powstania w getcie warszawskim, podarował prezydentowi Kielc Wojciechowi Lubawskiemu. Odnaleziono ją na śmietniku na osiedlu Szydłówek - W sobotę późnym popołudniem wracaliśmy do swojego mieszkania na osiedlu Szydłówek w Kielcach. Kiedy wysiedliśmy z samochodu, moja dziewczyna Katarzyna zauważyła, że przed śmietnikiem leżą książki - opowiada kielczanin Jakub Kotula. Podeszła, przejrzała i dwie zabrała ze sobą. - Jedna z nich to była książka <Entity name="Marek Edelman" type="person" category="lekarz">Marka Edelmana</Entity> "I była miłość w getcie". Katarzyna nie mogła uwierzyć, że ktoś mógł wyrzucić ją na śmietnik - mówi. Ich zdziwienie było jeszcze większe, gdy zobaczyli dedykację: "Panu Prezydentowi Wojciechowi Lubawskiemu z pełnym uznaniem i pozdrowieniami". A pod spodem podpis <Entity name="Marek Edelman" type="person

### Sprawdzanie zawodów w Wikipedii

#### Słownik z zawodami (zrobiony recznie)

In [36]:
import codecs

def read_jobs(file):
    l = []
    with codecs.open(file , "r", encoding = 'UTF-8') as f:
        l = f.read().split('\n')
    return l

In [37]:
folder = 'podzawody'
jobs = dict()
keys = read_jobs('zawody.txt')
for key in keys:
    jobs[key] = set(read_jobs(folder + '/' + key + '.txt'))

#### Morfeusz + wikipedia

In [2]:
import wikipedia
import morfeusz2
wikipedia.set_lang("pl")

In [47]:
def exclude_job_for_person(person):
    # looks for one of jobs in wikipedia summary page
    global jobs
    profession = []
    morf = morfeusz2.Morfeusz()
    try:
        page = wikipedia.page(person)
        l = []
        analysis = morf.analyse(page.summary)
        for i, j, interp in analysis:
            # Interp[0] - word in page.summary
            # Interp[1] - The base form of Interp[0] (with some tags or clear)
            #print(interp) #uncomment to see
            l.append(interp[0])
            l.append(interp[1])
        simple_words = set(l)
        for key in jobs.keys():
            if jobs[key].intersection(simple_words) != set():
                profession.append(key)
        if profession == []:
            profession.append('None')
    except:
        profession.append('None')
    return profession

In [52]:
people = ['Roman Giertych', 'Donald Tusk', 'Jarosław Kaczyński', 'Janusz Korwin-Mikke',
         'Barbara Kurdej-Szatan', 'Robert Mazurek', 'Roman Dmowski', 'Tomasz Kłos',
         'Andrzej Grabowski', 'Juliusz Słowacki']
for human in people:
    print(human + ' : ' + str(exclude_job_for_person(human)))

Roman Giertych : ['politycy', 'prawnicy']
Donald Tusk : ['politycy']
Jarosław Kaczyński : ['politycy', 'prawnicy']
Janusz Korwin-Mikke : ['politycy', 'dziennikarze']
Barbara Kurdej-Szatan : ['aktorzy', 'muzycy']
Robert Mazurek : ['dziennikarze']
Roman Dmowski : ['politycy', 'dziennikarze']
Tomasz Kłos : ['sportowcy']
Andrzej Grabowski : ['aktorzy', 'muzycy']
Juliusz Słowacki : ['poeci']


##### Proby uzyskania podstawowej wersji wyrażenia (nieudane)

In [31]:
persons = ['Kasi Kowalskiej', 'Roberta Lewandowskiego', 'Ania Lewandowska', 'Donaldem Tuskiem', 'Januszem Korwin-Mikke']

In [33]:
person = 'Kasi Kowalskiej'
for person in persons:
    morf = morfeusz2.Morfeusz()
    #page = wikipedia.page(person)
    analysis = morf.analyse(person)
    l = []
    for i, j, interp in analysis:
        if interp[2].split(':')[0] == 'subst':# or interp[2].strip(':')[0] == 'subst':
            l.append(interp[1])
        print(set(l))
    l = []

{'Kasia'}
{'Kasia'}
{'Kasia'}
{'Kasia'}
{'Kasia'}
{'Kasia'}
{'Kasia', 'Kowalska'}
{'Kasia', 'Kowalska'}
{'Robert:s2'}
{'Robert:s2'}
{'Robert:s2', 'Roberta'}
{'Robert:s2', 'Robert:s1', 'Roberta'}
{'Robert:s2', 'Robert:s1', 'Roberta', 'Roberto'}
{'Roberto', 'Lewandowski', 'Robert:s2', 'Roberta', 'Robert:s1'}
{'Ania'}
{'Ania', 'Lewandowska'}
{'Ania', 'Lewandowska'}
{'Donald'}
{'Donald', 'Tusk:s1'}
{'Janusz:s2'}
{'Janusz:s2', 'Janusz:s1'}
{'Janusz:s2', 'Janusz:s1', 'Korwin:s1'}
{'Janusz:s2', 'Janusz:s1', 'Korwin:s1'}
{'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1'}
{'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1', 'Korwin:s3'}
{'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1', 'Korwin:s3'}
{'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1', 'Korwin:s3'}
{'Mikke:s1', 'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1', 'Korwin:s3'}
{'Mikke:s1', 'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Korwin:s1', 'Korwin:s3'}
{'Mikke:s1', 'Janusz:s2', 'Korwin:s2', 'Janusz:s1', 'Mikke:s2', 'Korwin:s1

In [23]:
person = 'Kasi Kowalskiej'
morf = morfeusz2.Morfeusz()
#page = wikipedia.page(person)
analysis = morf.analyse(person)
l = []
for i, j, interp in analysis:
    print(interp)

('Kasi', 'Kasia', 'subst:sg:gen:f', ['imię'], [])
('Kasi', 'Kasia', 'subst:sg:dat.loc:f', ['imię'], [])
('Kasi', 'Kasia', 'subst:pl:gen:f', ['imię'], ['hom.', 'rzad.'])
('Kowalskiej', 'kowalski', 'adj:sg:dat:f:pos', [], [])
('Kowalskiej', 'kowalski', 'adj:sg:gen:f:pos', [], [])
('Kowalskiej', 'kowalski', 'adj:sg:loc:f:pos', [], [])
('Kowalskiej', 'Kowalska', 'subst:sg:gen:f', ['nazwisko'], [])
('Kowalskiej', 'Kowalska', 'subst:sg:dat.loc:f', ['nazwisko'], [])
